# 💾 에이전트 대화 저장 및 재개

> **개요**  
> AgentThread를 저장소에 저장하고 나중에 다시 불러와서 대화를 이어가는 방법을 알아봅니다.

## 🎯 왜 필요한가요?

에이전트를 서비스 또는 클라이언트 애플리케이션에서 호스팅할 때, 여러 요청이나 세션에 걸쳐 **대화 상태를 유지**해야 하는 경우가 많습니다. AgentThread를 저장하면 대화 컨텍스트를 나중에 다시 불러올 수 있습니다.

---

## 🚀 에이전트 생성 및 스레드 시작

먼저 에이전트를 생성하고 대화 상태를 저장할 새 스레드를 만듭니다.

In [ ]:
from azure.identity import DefaultAzureCredential
from agent_framework import ChatAgent
from agent_framework.azure import AzureOpenAIChatClient

agent = ChatAgent(
    chat_client=AzureOpenAIChatClient(credential=DefaultAzureCredential()),
    name="Assistant",
    instructions="당신은 한국어로 답변하는, 도움이 되는 어시스턴트입니다. 🤖✨"
)

thread = agent.get_new_thread()

---

## 💬 에이전트 실행

에이전트를 실행할 때 스레드를 전달하여 `AgentThread`가 대화 내용을 포함하게 합니다.

In [ ]:
# 에이전트를 실행하고 대화를 스레드에 추가합니다.
response = await agent.run("거인에 대한 짧은 농담을 해줘.", thread=thread)
print(response.text)

---

## 💿 스레드를 저장소에 저장하기

### 📝 직렬화 프로세스

1. 스레드의 `serialize` 메서드를 호출하여 데이터를 딕셔너리로 직렬화
2. JSON으로 변환
3. 데이터베이스, Blob 스토리지 또는 파일에 저장

In [ ]:
import json
import tempfile
import os

# 스레드 상태 직렬화
serialized_thread = await thread.serialize()
serialized_json = json.dumps(serialized_thread)

# 예시, 현재 경로에 agent_thread.json 파일로 저장하기
# 실제 프로덕션에서는 DB나 Blob 스토리지로 교체하는 것을 권장
current_dir = os.getcwd()
# temp_dir = tempfile.gettempdir()
print(f"현재 작업 디렉토리: {current_dir}")

file_path = os.path.join(current_dir, "agent_thread.json")
with open(file_path, "w") as f:
    f.write(serialized_json)

---

## 📂 스레드를 저장소에서 불러오기

### 🔄 역직렬화 프로세스

저장소에 저장된 대화를 다시 불러오려면, 저장된 JSON 데이터를 불러와서 `AgentThread` 인스턴스를 다시 생성합니다.

> **⚠️ 중요 사항**  
> 스레드는 **에이전트 인스턴스를 사용하여 역직렬화**되어야 하며, 사용되는 에이전트 유형은 원래 스레드를 생성할 때 사용한 것과 **동일**해야 합니다.

### 💡 왜 동일한 에이전트 유형이 필요한가요?

각 에이전트마다 고유한 스레드 유형이 있을 수 있으며, 해당 에이전트 유형에 특화된 추가 기능을 사용하여 스레드를 생성할 수 있기 때문입니다.

In [ ]:
# 저장된 JSON 불러오기
with open(file_path, "r") as f:
    loaded_json = f.read()

reloaded_data = json.loads(loaded_json)

# 복원한 스레드를 동일한 에이전트 유형에 연결된 AgentThread로 역직렬화
resumed_thread = await agent.deserialize_thread(reloaded_data)

---

## ▶️ 대화 재개

불러온 스레드를 사용하여 대화를 자연스럽게 이어갈 수 있습니다.

In [ ]:
# 복원된 스레드를 사용하여 대화를 계속합니다.
response = await agent.run("너는 재미있는 이야기꾼이구나", thread=resumed_thread)
print(response.text)

---

## 🎓 실용적인 멀티턴 예제

다음은 여러 상호작용에 걸쳐 컨텍스트를 유지하는 방법을 보여주는 완전한 예제입니다:

### 📋 실습 시나리오

1. **첫 번째 상호작용** - 사용자 이름 저장
2. **두 번째 상호작용** - 에이전트가 이름 기억
3. **직렬화** - 스레드를 저장소에 저장
4. **역직렬화** - 저장된 스레드 불러오기
5. **컨텍스트 확인** - 이전 대화 내용 기억 여부 확인

In [ ]:
from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient
from azure.identity import AzureCliCredential

agent = ChatAgent(
    chat_client=AzureAIAgentClient(credential=AzureCliCredential()),
    name="Assistant",
    instructions="당신은 한국어로 답변하는, 도움이 되는 어시스턴트입니다. 🤖✨"
)

# 지속적인 대화를 위해서 스레드 생성
thread = agent.get_new_thread()

# 첫번째 상호작용
response1 = await agent.run("내 이름은 마이클이야", thread=thread)
print(f"Agent: {response1.text}")
print("=" * 70)

# 두번째 상호작용 - 에이전트가 이름을 기억함
response2 = await agent.run("내 이름이 뭐였지?", thread=thread)
print(f"Agent: {response2.text}")  # Should mention "Alice"
print("=" * 70)

# 스토리지에 스레드 직렬화
serialized = await thread.serialize()

# 역직렬화하고 대화를 계속합니다.
new_thread = await agent.deserialize_thread(serialized)
response3 = await agent.run("우리가 무슨 이야기를 했지?", thread=new_thread)
print(f"Agent: {response3.text}")  # 이전 컨텍스트를 기억해야 함
